# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Изучение данных

In [167]:
#подгружаем необходимые для работы библиотеки
import pandas as pd

from pymystem3 import Mystem
m = Mystem()

In [168]:
#читаем файл методом read_csv() и знакомимся с общей информацией
try:
    data = pd.read_csv("data.csv")
except:
    data = pd.read_csv("/datasets/data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [169]:
#выводим на экран первые 15 строк датасета
display(data.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [170]:
#смотрим уникальные значения столбца "children"
data.groupby("children")["children"].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [171]:
#ознакамливаемся с граничными показаниями столбца "days_employed"
display(data["days_employed"].max())
display(data["days_employed"].min())

401755.40047533

-18388.949900568383

In [172]:
#смотрим уникальные значения столбца "education"
data["education"].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [173]:
#смотрим уникальные значения столбца "family_status"
data["family_status"].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [174]:
#смотрим уникальные значения столбца "income_type"
data["income_type"].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [175]:
#ознакамливаемся с граничными показаниями столбца "total_income	"
display(data["total_income"].max())
display(data["total_income"].min())

2265604.028722744

20667.26379327158

**Вывод**

Т.к. проект можно выполнять как на домашнем компьютере, так и в окружении практикума процесс считывания файла поместили в связку try-except, чтобы датасет мог считываться в любом случае и программа не выдавала ошибку. 

В датафрейме 21525 строк в 12 колонках. Используются числовые и строковые типы данных.

Из информацию к проекту мы знаем, что означают столбцы датасета:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

После первичного ознакомления с данными таблицы можно увидеть некоторые проблемы в данных, а именно: 
- в столбце **"days_employed"** и  **"total_income"** есть пропущенные значения типа NaN. Также столбец **"days_employed"**, в котором указан стаж работы заемщика в днях логично перевести из вещественного типа в целочисленный. 
- в столбце **"education"** использован верхний и нижний регистр из-за чего некоторые виды образования дублируются, 
- в столбце **"purpose"** данные, по всей вероятности вносились вручную, поэтому необходимо будет провести лемматизацию, для того чтобы разбить цели, на которые брался кредит по группам. 

Если копать глубже то вызывает вопрос почему большинство значений в столбце **"days_employed"** отрицательные. Возможно это ошибка выгрузки, как и наличие неправдоподобного стажа в 401755дней, что приблизительно равно 1116годам работы. Но т.к.этот столбец не является основным для выполнения проекта можно не применять к нему метод `abs()` и не вычленять артефакты.
Еще ошибки есть в столбце **"children"**, т.к.вызывает сомнение наличие в семье 20 детей(встречается 76 раз) или значение "-1"(встречается 47 раз). Считаю что эти ошибки возникли при вводе данных, но нам интересно именно наличие или отсутствие детей, поэтому оставим эти данные без изменения.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [176]:
#пропуска в столбцах "days_employed" и "total_income" заменяем средним и медианным значением соответственно
data["days_employed"] = data["days_employed"].fillna(data["days_employed"].mean())
data["total_income"] = data["total_income"].fillna(data.groupby("income_type")["total_income"].transform("median"))

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

   Пропуска в столбцах **"days_employed"** и **"total_income"** составляют больше 2тысяч строк, что равно приблизительно 10% от всего массива данных. Удалять такой большой кусок информации нецелесообразно, поэтому применяем метод `fillna()` для заполнения пустых ячеек. В столбце **"days_employed"** заполнили пропуски средними значениями, а в столбце **"total_income"** заменив их медианными значениями для каждой из групп по типу занятости, т.к.разброс дохода у безработного и допустим предпринимателя может быть значительным.

### Замена типа данных

In [177]:
#переводим столбец "days_employed" в целочисленный формат
data["days_employed"] = data["days_employed"].astype("int")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

С помощью метода `astype()` с аргументом `int` меняем тип данных в столбце **"days_employed"** на целочисленный.

### Обработка дубликатов

In [178]:
#приводим данные в столбце "education" к нижнему регистру
data["education"] = data["education"].str.lower()

In [179]:
#проверяем датасет на наличие явных дубликатов
data.duplicated().sum()

71

In [180]:
#удаляем явные дубликаты и делаем переиндексацию строк
data = data.drop_duplicates().reset_index(drop=True)
data.shape

(21454, 12)

**Вывод**

Проверка явных дубликатов показала их в количестве 71. Так как 71 это ничтожно малое число из выборки в 21525 строк мы можем себе позволить избавиться от этих дубликатов. Методом `drop_duplicates()` избавляемся от них. Если бы количество дубликатов угрожало точности исследования необходимо было бы разбираться что это: ошибка выгрузки или человеческий фактор и исправлять данные. Методом `reset_index(drop=True)` переиндексируем строки датафрейма.

### Лемматизация

In [181]:
#пишем функцию для лемматизации столбца с целями кредита
def lemma(row):
    lemmas_purpose = m.lemmatize(row["purpose"])
    if "автомобиль" in lemmas_purpose:
        return "автомобиль"
    if ("жилье" in lemmas_purpose) or ("недвижимость" in lemmas_purpose):
        return "недвижимость"
    if "свадьба" in lemmas_purpose:
        return "свадьба"
    if "образование" in lemmas_purpose:
        return "образование"


**Вывод**

Еще при первоначальном ознакомлении с таблицей было замечено, что столбец с целями на которые берется кредит заполнен очень разрозненной информацией. Для того чтобы выделить основные категории целей необходимо провести лемматизацию. Так как лемматизировать нам нужно целый столбец напишем свою функцию, которая будет перебирать значения в столбце **"purpose"** и возвращать лемматизированные формы слова. 

### Категоризация данных

In [182]:
#применяем функцию для категоризации целей и создания нового столбца в датафрейме
data["purpose_category"] = data.apply(lemma,axis=1)

In [183]:
#пишем и применяем функцию для категоризации семей с детьми и без
def child (row):
    if row == 0:
        return "бездетные"
    else:
        return "есть дети"

data["children_category"] = data["children"].apply(child)

In [184]:
#пишем функцию, которая разделит заемщиков на категории по уровню дохода
def income_level (row):
    if row <=36000:
        return "бедные и выше бедности"
    if (row > 36000) and (row <=150000):
        return "предсредний класс"
    if (row > 150000) and (row <=250000):
        return "средний класс"
    if (row > 250000) and (row <=500000):
        return "выше среднего"
    else:
        return "богатые"

data["income_level"] = data["total_income"].apply(income_level)

display(data.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,income_level
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,есть дети,выше среднего
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,есть дети,предсредний класс
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,бездетные,предсредний класс
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,есть дети,выше среднего
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,бездетные,средний класс
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость,бездетные,выше среднего
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость,бездетные,средний класс
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование,бездетные,предсредний класс
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба,есть дети,предсредний класс
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость,бездетные,предсредний класс


**Вывод**

Для того чтобы ответить на вопросы исследования, а именно как влияют цели кредита, наличие детей и уровень дохода на возврат кредита в срок проводим категоризацию соответствующих колонок. В дальнейшем это поможет удобно их группировать и проводить расчеты. Cоздадим новый столбец **"purpose_category"**, в который благодаря методу `apply()` будут попадать категоризированые цели кредита. Далее создадим функцию, которая распределит заемщиков на тех кто имеет детей и кто нет. 

С тем как категоризировать цели и наличие детей вопросов не возникает, но в исходных данных не указано, на какие категории по уровню дохода необходимо разделить заемщиков. В идеале нужно обратиться к заказчику исследования и уточнить какие категории выделять. Но т.к.в учебном проекте мы не коммуницируем с заказчиком будем опираться на распределение указаное в статье [Яндекс.Дзен](https://zen.yandex.ru/media/dogecrypto/kak-poschitat-k-kakomu-klassu-vy-otnosites-po-urovniu-dohoda-5b150b5aea0fe700a848e1b1) сократив количество категорий до 5.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [185]:
#создадим новую таблицу для работы с зависимостью возврата кредита от наличия детей
children_and_debt = data[["children","children_category","debt"]]

#сделаем сводную таблицу по категориям
children_and_debt_pivot = children_and_debt.pivot_table(index="children_category",columns="debt",values="children", aggfunc='count')

#выясним какой процент заемщиков имеет задолженность по каждой категории
children_and_debt_pivot["percent_debt"] = (children_and_debt_pivot[1] / (children_and_debt_pivot[0] + children_and_debt_pivot[1])) * 100

#переименуем столбцы для наглядности
children_and_debt_pivot.set_axis(["не было задолженности","была задолженность","процент заемщиков с задолженностью"],axis="columns",inplace=True)

#отформатируем колонку с процентами, оставив 2 знака после запятой
children_and_debt_pivot.style.format({"процент заемщиков с задолженностью":"{:.2f}%"})


,не было задолженности,была задолженность,процент заемщиков с задолженностью
children_category,,,
бездетные,13028,1063,7.54%
есть дети,6685,678,9.21%


**Вывод**

Предположим что в столбце **"debt"** 0 - принимает значение если заемщик не имел задолженности по кредиту, а 1 - если имел. Хотя эту информацию следует уточнять у разработчиков или искать в документации. Тогда путем вычислений мы можем определить что среди семей с детьми процент должников выше чем у семей без детей. Как мне кажется целесообразно было бы посмотреть как меняется этот показатель еще и от количества детей, а не просто от их наличия. Но так как по заданию нам необходимо определить зависимость от наличия детей впринципе, то произведенных манипуляций достаточно.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [186]:
#создадим новую таблицу для работы с зависимостью возврата кредита от наличия детей
family_and_debt = data[["family_status","family_status_id","debt"]]

#сделаем сводную таблицу по категориям
family_and_debt_pivot = family_and_debt.pivot_table(index="family_status",columns="debt",values="family_status_id", aggfunc='count')

#выясним какой процент заемщиков имеет задолженность по каждой категории
family_and_debt_pivot["percent_debt"] = (family_and_debt_pivot[1] / (family_and_debt_pivot[0] + family_and_debt_pivot[1])) * 100

#переименуем столбцы для наглядности
family_and_debt_pivot.set_axis(["не было задолженности","была задолженность","процент заемщиков с задолженностью"],axis="columns",inplace=True)

#отформатируем колонку с процентами, оставив 2 знака после запятой
family_and_debt_pivot.style.format({"процент заемщиков с задолженностью":"{:.2f}%"})


,не было задолженности,была задолженность,процент заемщиков с задолженностью
family_status,,,
Не женат / не замужем,2536,274,9.75%
в разводе,1110,85,7.11%
вдовец / вдова,896,63,6.57%
гражданский брак,3763,388,9.35%
женат / замужем,11408,931,7.55%


**Вывод**

По сводной таблице можно сказать, что наибольший процент невернувших кредит у заемщиков со статусом "Не женат / не замужем" и "гражданский брак", а наименьший у вдовцов. Это можно связать с возрастом заемщиков.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [187]:
#создадим новую таблицу для работы с зависимостью возврата кредита от наличия детей
income_and_debt = data[["income_level","total_income","debt"]]

#сделаем сводную таблицу по категориям
income_and_debt_pivot = income_and_debt.pivot_table(index="income_level",columns="debt",values="total_income", aggfunc='count')

#выясним какой процент заемщиков имеет задолженность по каждой категории
income_and_debt_pivot["percent_debt"] = (income_and_debt_pivot[1] / (income_and_debt_pivot[0] + income_and_debt_pivot[1])) * 100

#переименуем столбцы для наглядности
income_and_debt_pivot.set_axis(["не было задолженности","была задолженность","процент заемщиков с задолженностью"],axis="columns",inplace=True)

#отформатируем колонку с процентами, оставив 2 знака после запятой
income_and_debt_pivot.style.format({"процент заемщиков с задолженностью":"{:.2f}%"})


,не было задолженности,была задолженность,процент заемщиков с задолженностью
income_level,,,
бедные и выше бедности,71,4,5.33%
богатые,208,14,6.31%
выше среднего,2411,180,6.95%
предсредний класс,10574,974,8.43%
средний класс,6449,569,8.11%


**Вывод**

По данным таблицы видно, что наибольший процент среди задолженников у предсреднего и среднего класса, что и понятно так как на долю этих категорий приходится наибольшее количество значений. Что показательно люди с наименьшим доходом имеют меньший процент просрочек, но количество данных не позволяет делать из этого каких-то выводов, т.к.в нашем исследовании всего 75 семей попали в эту категорию.

- Как разные цели кредита влияют на его возврат в срок?

In [188]:
#создадим новую таблицу для работы с зависимостью возврата кредита от наличия детей
purpose_and_debt = data[["purpose_category","purpose","debt"]]

#сделаем сводную таблицу по категориям
purpose_and_debt_pivot = purpose_and_debt.pivot_table(index="purpose_category",columns="debt",values="purpose", aggfunc='count')

#выясним какой процент заемщиков имеет задолженность по каждой категории
purpose_and_debt_pivot["percent_debt"] = (purpose_and_debt_pivot[1] / (purpose_and_debt_pivot[0] + purpose_and_debt_pivot[1])) * 100

#переименуем столбцы для наглядности
purpose_and_debt_pivot.set_axis(["не было задолженности","была задолженность","процент заемщиков с задолженностью"],axis="columns",inplace=True)

#отформатируем колонку с процентами, оставив 2 знака после запятой
purpose_and_debt_pivot.style.format({"процент заемщиков с задолженностью":"{:.2f}%"})

,не было задолженности,была задолженность,процент заемщиков с задолженностью
purpose_category,,,
автомобиль,3903,403,9.36%
недвижимость,10029,782,7.23%
образование,3643,370,9.22%
свадьба,2138,186,8.00%


**Вывод**

Наибольший процент невозвратов у кредитов взятых на покупку автомобиля и на образование. Я бы связала это как и в случае с семейным статусом с возрастом заемщика, на образование в основном берут кредиты молодые студенты.

## Шаг 4. Общий вывод

В процессе проведения исследования мы сделали предобработку данных, сохранили строки с пропущенными значениями и удалили всего 71 дубликат, т.е.сохранили основной массив информации. Также для удобства пользования информацией воспользовались лемматизацией и категоризировали некоторые данные.

Отвечая на поставленные в исследовании вопросы было замечено следующее:
- бездетные заемщики реже просрочивают платежи;
- наибольший процент невернувших кредит у заемщиков со статусом "Не женат / не замужем" и "гражданский брак";
- наибольший процент среди задолженников у предсреднего и среднего класса по уровню дохода;
- у кредитов, взятых на покупку автомобиля или образование, больше вероятность быть просроченным. 
